In [2]:
from langchain import SQLDatabase
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
import os 
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from prompts.templates import *

In [ ]:
### DEFINE ALL MODEL AND MEMORY INSTANCES HERE
load_dotenv()
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")
DB_URI= os.getenv("DB_URI")
db = SQLDatabase.from_uri(DB_URI)
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
embbeder=OpenAIEmbeddings(model='text-embedding-3-small', dimensions=768)
str_output_parser = StrOutputParser()
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)

In [11]:
### FUNCTION TO GET CURRENT CONTEXT
def get_current_context(memory: ConversationBufferMemory):
    history_str=memory.buffer_as_str
    summarize_chain= history_summarization_template | llm | str_output_parser
    summary=summarize_chain.invoke({"input":history_str})
    memory.clear()
    memory.save_context(inputs={"input":summary},outputs={"output":""})
    return memory

In [5]:
### FUNCTION TO GET THE MOST SIMILAR NEWS ARTICLES TO THE ENTITIES PROVIDED
def find_top_k_similar(entities:str):
    embedding=embbeder.embed_query(entities)
    query= f"SELECT title, url, description, content, cat, subcat, published_date, author FROM news ORDER BY embedding <#> '{str(embedding)}' LIMIT 5"
    return db.run(query)
    

In [6]:
chain= entites_extraction_template | llm | str_output_parser | find_top_k_similar | str_output_parser| sql_response_parsing_template | llm | str_output_parser

In [12]:

# Example interaction function
def run_agent(user_input,memory):
    # Get chat history and summarize it
    memory = get_current_context(memory)

    response=chain.invoke({"input": user_input, "context": memory.buffer_as_str})
    
    memory.save_context({"input": user_input}, {"response": response})
    
    return response



In [20]:
user_query = "Continue giving me articles with the same topic as the previous one"
response = run_agent(user_query,memory)
print(response)

1. **Đại học Quốc gia Hà Nội hợp tác với 2 đại học hàng đầu Trung Quốc**: Hà Nội sẽ hợp tác với ĐH Bắc Kinh và ĐH Thanh Hoa về đào tạo khoa học trẻ, trao đổi giảng viên, sinh viên trong các lĩnh vực công nghệ, kỹ thuật, trí tuệ nhân tạo, bán dẫn, và khoa học xã hội nhân văn. [Xem thêm](https://dantri.com.vn/giao-duc/dai-hoc-quoc-gia-ha-noi-bat-tay-voi-2-dai-hoc-hang-dau-trung-quoc-20240830160703959.htm)

2. **Điểm chuẩn Đại học Khoa học Xã hội và Nhân văn Hà Nội**: Ngành Quan hệ công chúng có điểm chuẩn cao nhất là 29,1, với trung bình 9,7 điểm mỗi môn. [Xem thêm](https://vnexpress.net/diem-chuan-dai-hoc-khoa-hoc-xa-hoi-va-nhan-van-ha-noi-cao-nhat-29-1-4782335.html)

3. **Đại học Khoa học và Công nghệ Hà Nội**: Trường chỉ tuyển sinh khoảng 1.000 sinh viên mỗi năm, định hướng đào tạo nguồn nhân lực khoa học - công nghệ cao chuẩn quốc tế. [Xem thêm](https://vnexpress.net/dai-hoc-dao-tao-nhan-luc-khoa-hoc-cong-nghe-theo-chuan-quoc-te-4775409.html)


In [21]:
print(memory.buffer_as_str)

Human: The user requested articles related to Hanoi University of Science and Technology, and the chatbot provided information on journalism education and high admission scores. The conversation then transitioned to discussing chicken eggs, including their nutritional benefits and preferences in Vietnam. The user continued to seek more articles on similar topics, prompting the chatbot to share various educational articles.
AI: 
Human: Continue giving me articles with the same topic as the previous one
AI: 1. **Đại học Quốc gia Hà Nội hợp tác với 2 đại học hàng đầu Trung Quốc**: Hà Nội sẽ hợp tác với ĐH Bắc Kinh và ĐH Thanh Hoa về đào tạo khoa học trẻ, trao đổi giảng viên, sinh viên trong các lĩnh vực công nghệ, kỹ thuật, trí tuệ nhân tạo, bán dẫn, và khoa học xã hội nhân văn. [Xem thêm](https://dantri.com.vn/giao-duc/dai-hoc-quoc-gia-ha-noi-bat-tay-voi-2-dai-hoc-hang-dau-trung-quoc-20240830160703959.htm)

2. **Điểm chuẩn Đại học Khoa học Xã hội và Nhân văn Hà Nội**: Ngành Quan hệ công 